In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import math
import gensim
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile
import collections

# Movie Review Data Cleaning

In [2]:
def read_raw(file_name, label):
  with open(file_name, 'r') as file:
    data = []
    for line in file:
        processed_line = preprocess_raw(line)
        data.append([processed_line.split(), label])
  return data

def preprocess_raw(text):
  text = text.replace('\u202f', ' ').replace('\xa0', ' ')
  out = ''
  for i, char in enumerate(text.lower()):
    if char.isalpha()==True or char==' ':
      out += char
  out=" ".join(out.split())
  return out

def build_data(direc):
  data=[]
  counter=0
  for label in ['pos', 'neg']:
    label_direc = os.path.join(direc, label)
    for filename in os.listdir(label_direc):
      if (label == 'pos'):
        data += read_raw(os.path.join(label_direc,str(filename)), 1)
      else:
        data += read_raw(os.path.join(label_direc,str(filename)), 0)
      counter += 1
      print(counter)
  print('Done')
  return data

In [117]:
data = build_data('./drive/My Drive/txt_sentoken/')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Data to File

In [0]:
data = np.array(data)
file = open('data', 'wb')
pickle.dump(data, file)
file.close()

# Movie Review Dataset

In [3]:
class MovieReviewDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.data = dataset
        self.transform = transform
    
    def __getitem__(self, idx):
        temp = self.data[idx]
        if (self.transform):
            temp = self.transform(sample)
        
        sample = {'sentence': temp[0] ,'label': temp[1]}
        return sample
    
    def __len__(self):
        return len(self.data)            

# Load data file

In [4]:
file = open('data', 'rb')
data = pickle.load(file)
file.close()

64720


# Train Embedding Model

In [8]:
embed_model = Word2Vec(size=300, min_count=1)
embed_model.build_vocab(data[:,0])
total_examples = embed_model.corpus_count
pretrained = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
embed_model.build_vocab([list(pretrained.vocab.keys())], update=True)
embed_model.intersect_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True, lockf=1.0)
embed_model.train(data[:,0], total_examples=total_examples, epochs=embed_model.iter)



(5965743, 6767065)

# Save Trained Embedded Model

In [17]:
embed_model.wv.save('embedding_model')

# Load and Test saved model

In [6]:
embed_model = KeyedVectors.load('embedding_model')

In [7]:
sent = 'this is a sentence januarys' 
print([embed_model.wv[word] for word in sent.split()])

51e-01, -1.96436739e+00,
        1.61268890e+00,  1.25774097e+00,  2.67858338e+00, -3.90290886e-01,
       -3.19548905e-01, -2.38709497e+00,  2.79759359e+00,  1.06206644e+00,
        1.00251186e+00, -3.74796033e-01, -4.76959378e-01, -3.94884586e+00,
       -3.09618801e-01,  2.02921343e+00,  1.13458431e+00,  1.91048694e+00,
        1.00524565e-02, -2.71796608e+00, -6.43906295e-01,  2.05248952e+00,
        1.54985034e+00,  3.15263796e+00,  2.27099586e+00, -1.32776594e+00,
        2.67235446e+00, -7.18834221e-01, -3.72465062e+00, -2.90466857e+00,
       -3.52452129e-01,  2.84353167e-01, -2.99009848e+00,  1.37953115e+00,
        7.81196296e-01,  3.55695891e+00,  4.70857918e-01, -1.06122994e+00,
       -4.46460962e+00, -7.07270622e-01,  1.51226270e+00,  4.68177414e+00,
        4.16328430e+00,  6.38371468e-01,  2.62311387e+00, -2.40969920e+00,
       -2.40465832e+00, -2.19027686e+00, -1.92083633e+00,  5.26654661e-01,
       -7.03705996e-02,  6.78789020e-01,  1.66139090e+00,  3.49503493e+00,


In [30]:
class GoogleEmbedding(nn.Module):
  def __init__(self, model_path):
    super(GoogleEmbedding, self).__init__()

    # Load Google's pre-trained Word2Vec model.
    self.model = KeyedVectors.load(model_path, mmap='r')

  def forward(self, batch):
    out = []
    for sent in batch:
      sent_embed = []
      for word in sent:
        sent_embed.append(self.model[word])
      
      sent_embed = np.array(sent_embed)
      out.append(sent_embed)
    return np.array(out)

In [31]:
embedding = GoogleEmbedding('embedding_model')

In [38]:
a = embedding([['this' ,'is', 'one'], ['this', 'is', 'two']])
print(a.shape)

(2, 3, 300)


In [0]:
MIN_FREQ=30
MAX_LEN=50

In [0]:
class Vocab():
  def __init__(self, name, tokens, min_freq):
    self.name = name
    self.index2word = {
      0: 'pad',
      1: 'unk'
    }
    self.word2index = {v: k for k, v in self.index2word.items()}
    self.num_word = 2
    token_freq = collections.Counter(tokens)
    tokens = [token for token in tokens if token_freq[token] >= MIN_FREQ]
    self._build_vocab(tokens)
    
  def _build_vocab(self, tokens):
    for token in tokens:
      if token not in self.word2index:
        self.word2index[token] = self.num_word
        self.index2word[self.num_word] = token
        self.num_word += 1
        
  def __getitem__(self, tokens):
    if not isinstance(tokens, (list, tuple)):
      return self.word2index.get(tokens, self.word2index['unk'])
    else:
      return [self.__getitem__(token) for token in tokens]

In [0]:
def build_vocab(name, tokens, min_freq):
  sentences=tokens[:,0]
  tokens = [token for sentence in sentences for token in sentence]
  return Vocab(name, tokens, min_freq)

def build_vocabs(lang_src, src_text):
  vocab_src = build_vocab(lang_src, src_text, MIN_FREQ)
  return vocab_src

def pad(line, padding_token):
  return line + [padding_token] * (MAX_LEN-len(line))

def build_tensor(text, reviews_vocab):
  labels=text[:,1]
  lines = [reviews_vocab[line] for line in text[:,0]]
  lst=[]
  for i in range(len(lines)):
    padded_sentence=torch.tensor(pad(lines[i],reviews_vocab['pad']))
    label=torch.tensor(labels[i])
    lst.append((padded_sentence,label))
  return lst

def load_reviews_data():
  reviews_vocab = build_vocabs('reviews', data)
  reviews_array = build_tensor(data, reviews_vocab)
  return reviews_array


#source, target = prepare_data(raw_text, max_len=MAX_LEN)
#vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size=2)
#print('Vocabulary size of source language: {}'.format(vocab_eng.num_word))
#print('Vocabulary size of target language: {}'.format(vocab_fra.num_word))
#print('Total number of sentence pairs: {}'.format(len(source)))

In [0]:
for i in range(len(data[:,0])):
  data[:,0][i]=data[:,0][i][:MAX_LEN]

In [163]:
reviews_array=load_reviews_data()
reviews_array[:,0]

(tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  1, 15,  1, 16, 17,
          1, 18, 19,  1, 17, 20,  1, 21, 22, 23, 24, 25, 26, 27, 28, 29,  5, 30,
         31,  4, 32, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]), tensor(1))

In [132]:
class CRAN(nn.Module):
  def __init__(self,vocab_size,embedding_size,cnn_window_length,hidden_units,p_dropout):
    super(CRAN,self).__init__()

    """
    inputs:
      cnn_num_filters: the number of convolutional kernels, represents the number of output channels parameter in Conv2d
      cnn_window_length: in a kernel size of d*l, d represents the embedding size and l represents the window length
      LSTM_hidden_units: number of hidden units in LSTM layer
      dropout: dropout probability for CNN
      embedding_size: length of embedded word vectors 
    """
    self.embedding=nn.Embedding(vocab_size,embedding_size)
    self.cnn=torch.nn.Conv2d(1,hidden_units,(cnn_window_length,embedding_size), padding=(1,0))
    self.dropout=torch.nn.Dropout(p_dropout)
    self.rnn=torch.nn.LSTM(embedding_size,hidden_units,1, batch_first=True)
    self.hidden_size = hidden_units
    self.dense = nn.Linear(hidden_units, 2)
  def forward(self,batch,labels):
    word_embedded=self.embedding(batch)
    (N,T,d)=batch.shape
    #apply convolutional filters to the input sentences
    cnn_output=self.cnn(batch.unsqueeze(1))
    #cnn_output will be of shape (N,cnn_num_filters,H_out from Pytorch documentation,1)
    shape=cnn_output.shape
    cnn_output= F.relu(cnn_output.view(N,shape[1],shape[2]))
    #average across the different filter outputs
    cnn_output=torch.mean(cnn_output,1)

    h = batch.new_zeros(1, N, self.hidden_size)
    c = batch.new_zeros(1, N, self.hidden_size)

    lstm_out, (h,c) = self.rnn(batch, (h, c))

    s = torch.mean(lstm_out * cnn_output.unsqueeze(2), 1)

    pred = F.log_softmax(self.dense(s))
    
    if (self.training):
      loss = F.nll_loss(pred, labels)

      return loss, pred.argmax(dim=-1)
    else:
      return pred

tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  1, 15,  1, 16, 17,
         1, 18, 19,  1, 17, 20,  1, 21, 22, 23, 24, 25, 26, 27, 28, 29,  5, 30,
        31,  4, 32, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [153]:
reviews_array

tensor([[  2,   3,   4,  ...,   0,   0,   0],
        [ 34,   1,  35,  ...,   0,   0,   0],
        [ 46,  56,  40,  ...,   0,   0,   0],
        ...,
        [209, 829, 272,  ...,   0,   0,   0],
        [  1,   1, 872,  ...,   0,   0,   0],
        [209, 829,   1,  ...,   0,   0,   0]])